In [5]:
from dotenv import load_dotenv
import os

load_dotenv('pwd.env')

db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv("DB_HOST")

In [6]:
from sqlalchemy import create_engine, text 

username = db_user
password = db_password
host = db_host
port = '5433'
database = 'companies_DWH'


connection_string = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

try:
    with engine.connect() as connection:
        print("connection was successful!")
except Exception as e:
    print("connection failed:", e)

connection was successful!


<h2>Merging tech_layoffs.csv files<h2>

In [7]:
import pandas as pd 

cleaned_tech_layoffs_q1 = pd.read_csv(r'D:\DEPI Data Engineering\personalWork\Company layoffs\cleaned_tech_layoffs_q1.csv', sep = ';')
cleaned_tech_layoffs_q2 = pd.read_csv(r'D:\DEPI Data Engineering\personalWork\Company layoffs\cleaned_tech_layoffs_q2.csv', sep = ',')

cleaned_tech_layoffs = pd.concat([cleaned_tech_layoffs_q1, cleaned_tech_layoffs_q2], axis = 0)
cleaned_tech_layoffs.to_csv('cleaned_tech_layoffs 2020 - 2024.csv', index = False)

print(cleaned_tech_layoffs)

      company_id         Company             Location_HQ Country  \
0              3       ShareChat               Bengaluru   India   
1              4       InSightec                   Haifa  Israel   
2              6  Enphase Energy  San Francisco Bay Area     USA   
3              7           Udaan               Bengaluru   India   
4             14          Cruise  San Francisco Bay Area     USA   
...          ...             ...                     ...     ...   
1834        5251    Moxion Power                Richmond     USA   
1835        4983          Perion                Tel Aviv  Israel   
1836        4740          Planet           San Francisco     USA   
1837        3570        RealPage                  Dallas     USA   
1838        5149       Priceline                 Norwalk     USA   

          Continent  Laid_Off Date_layoffs  Percentage  \
0              Asia     200.0   2023-12-20          15   
1              Asia     100.0   2023-12-19          20   
2     Nor

In [8]:
cleaned_tech_layoffs.to_sql('layoffs_companies', engine, if_exists='replace', index = False, schema='public')

511

<h2>conditional split<h2>

In [9]:
import pandas as pd

cleaned_companies_sorted = pd.read_csv(r'D:\DEPI Data Engineering\personalWork\Company layoffs\cleaned_companies_sorted.csv')

condition = cleaned_companies_sorted['total employee estimate'] >= 200000

large_size_companies = cleaned_companies_sorted[condition]
small_mid_size_companies = cleaned_companies_sorted[~condition]


In [10]:
large_size_companies.to_sql('large size companies', engine, if_exists = 'replace', index = False, schema = 'public')

small_mid_size_companies.to_sql('small & mid size companies', engine, if_exists = 'replace', index = False, schema = 'public')

846

In [11]:
try:
    cleaned_companies_sorted = pd.read_csv(r'D:\DEPI Data Engineering\personalWork\Company layoffs\cleaned_companies_sorted.csv')

    condition = cleaned_companies_sorted['total employee estimate'] >= 20000

    large_size_companies = cleaned_companies_sorted[condition]
    small_mid_size_companies = cleaned_companies_sorted[~condition]

    large_size_companies.to_sql('large_size_companies', engine, if_exists='replace', index=False, schema='public')
    small_mid_size_companies.to_sql('small_mid_size_companies', engine, if_exists='replace', index=False, schema='public')

    print("Data successfully split and stored in the database.")
except Exception as e:
    print(f"An error occurred: {e}")

Data successfully split and stored in the database.
